# Memory

A maioria das aplicações de Modelos de LLM possui uma interface conversacional. Um componente essencial de uma conversa é a capacidade de se referir a informações introduzidas anteriormente na conversa. No mínimo, um sistema conversacional deve ser capaz de acessar diretamente alguma janela de mensagens passadas. Um sistema mais complexo precisará ter um modelo de mundo que está constantemente atualizando, o que lhe permite fazer coisas como manter informações sobre entidades e suas relações.

Chamamos essa capacidade de armazenar informações sobre interações passadas de "Memory", ou memória. LangChain oferece muitas utilidades para adicionar memória a um sistema. Essas utilidades podem ser usadas por si só ou incorporadas de maneira integrada em uma chain.

In [1]:
from langchain_core.chat_history import InMemoryChatMessageHistory

memory = InMemoryChatMessageHistory()


In [2]:
memory.add_user_message('Olá, modelo!')
memory.add_ai_message('Olá, user')

In [3]:
memory.messages

[HumanMessage(content='Olá, modelo!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Olá, user', additional_kwargs={}, response_metadata={})]

## Criando uma conversa com memória

In [4]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "Você é um tutor de programação chamado Asimo. Responda as perguntas de forma didática."),
    ("placeholder", "{memoria}"),
    ("human", "{pergunta}"),
])
chain = prompt | ChatOpenAI()

In [5]:
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}
def get_by_session_id(session_id):
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

chain_com_memoria = RunnableWithMessageHistory(
    chain,
    get_by_session_id,
    input_messages_key='pergunta',
    history_messages_key='memoria'
)

In [6]:
config = {'configurable': {'session_id': 'usuaria_a'}}
resposta = chain_com_memoria.invoke({'pergunta': 'O meu nome é Adriano'}, config=config)
resposta

AIMessage(content='Olá, Adriano! Como posso te ajudar hoje?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 40, 'total_tokens': 55, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-ae30bcb9-e748-4071-8e45-c5014fcef6d9-0', usage_metadata={'input_tokens': 40, 'output_tokens': 15, 'total_tokens': 55, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [7]:
resposta = chain_com_memoria.invoke({'pergunta': 'Qual é o meu nome?'}, config=config)
resposta

AIMessage(content='Seu nome é Adriano. Como posso te ajudar hoje, Adriano?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 68, 'total_tokens': 87, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-4b120450-d4f3-43f7-8724-4568341fb977-0', usage_metadata={'input_tokens': 68, 'output_tokens': 19, 'total_tokens': 87, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [8]:
config = {'configurable': {'session_id': 'usuaria_b'}}
resposta = chain_com_memoria.invoke({'pergunta': 'Qual é o meu nome?'}, config=config)
resposta

AIMessage(content='Desculpe, mas não tenho acesso ao seu nome. Como posso te chamar? Como posso te ajudar hoje?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 40, 'total_tokens': 69, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-dcd34e3b-1a22-47f5-a5a4-7802ee53175f-0', usage_metadata={'input_tokens': 40, 'output_tokens': 29, 'total_tokens': 69, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})